In [1]:
import pandas as pd
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import numpy as np
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import TransformedTargetForecaster, DirectTimeSeriesRegressionForecaster, DirectTabularRegressionForecaster
from sktime.transformations.series.detrend import Deseasonalizer
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_squared_error
from sktime.forecasting.compose import make_reduction
from sklearn.exceptions import ConvergenceWarning
import requests
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.forecasting.compose import YfromX
from sktime.transformations.series.boxcox import LogTransformer
from sktime.transformations.compose import ColumnwiseTransformer
from sktime.forecasting.compose import ForecastingPipeline

import warnings
warnings.filterwarnings('ignore')

c:\Users\rkris\miniconda3\envs\slalomenv\Lib\site-packages\shap\utils\_clustering.py:34: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
c:\Users\rkris\miniconda3\envs\slalomenv\Lib\site-packages\shap\utils\_clustering.py:53: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
c:\Users\rkris\miniconda3\envs\slalomenv\Lib\site-packages\shap\utils\_clustering.py

In [2]:
price_old_df = pd.read_csv('https://raw.githubusercontent.com/slalom-ubc-mds/Power-Price-Prediction/main/data/processed/supply_load_price.csv', parse_dates=['Date (MST)'], index_col='Date (MST)')
price_old_df = price_old_df.asfreq('H')
price_old_df = price_old_df.sort_values(by='Date (MST)')
price_old_df = price_old_df['2022-12':]
price_df = price_old_df[['price']]

In [3]:
y = price_df
X = price_old_df.drop(columns=['price'])

In [4]:
from sklearn.preprocessing import StandardScaler
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor

direct_pipe_t = TransformedTargetForecaster(
                    [
                        (
                            "forecast",
                            make_reduction(
                                RandomForestRegressor(n_estimators=1, n_jobs=-1),
                                window_length=24,
                                strategy="direct",
                            ),
                        ),
                    ]
                )

fh = ForecastingHorizon(np.arange(1, 12+1))
direct_pipe_t.fit(y=y, X=X, fh = fh)


TransformedTargetForecaster(steps=[('forecast',
                                    DirectTabularRegressionForecaster(estimator=RandomForestRegressor(n_estimators=1, n_jobs=-1),
                                                                      window_length=24))])

In [5]:
import pandas as pd
def create_lagged_columns(X, lag_range=24):
    lagged_names = []
    for col in X:
        for lag in range(lag_range, 0, -1):
            lagged_names.append(f"{col}_lag{lag}")
    return lagged_names
labels = create_lagged_columns(['price'] + X.columns.values.tolist(), lag_range=24)
len(labels)

1200

In [6]:
y_transformed = direct_pipe_t.transform(y)  # does all the trasnformations on y
y_enc, X_enc = direct_pipe_t.forecaster_._transform(y_transformed, X)  # TransfomedTargetForecaster does not transform X, only y. 

In [7]:
array_df = pd.DataFrame(y_enc)
array_df.columns = ['price1', 'price2', 'price3', 'price4','price5', 'price6','price7', 'price8','price9', 'price10', 'price11', 'price12']
array_df

,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,price12
0,178.63,150.99,106.89,187.26,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82
1,150.99,106.89,187.26,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82,337.13
2,106.89,187.26,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82,337.13,420.30
3,187.26,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82,337.13,420.30,527.70
4,596.05,120.79,760.57,574.26,762.16,340.44,178.05,181.82,337.13,420.30,527.70,645.20
...,...,...,...,...,...,...,...,...,...,...,...,...
2863,51.07,49.35,47.94,49.26,47.61,48.56,46.05,46.53,46.03,43.55,46.25,50.86
2864,49.35,47.94,49.26,47.61,48.56,46.05,46.53,46.03,43.55,46.25,50.86,49.35
2865,47.94,49.26,47.61,48.56,46.05,46.53,46.03,43.55,46.25,50.86,49.35,51.28
2866,49.26,47.61,48.56,46.05,46.53,46.03,43.55,46.25,50.86,49.35,51.28,48.39


In [8]:
X_enc = pd.DataFrame(X_enc, columns=labels)

In [9]:
merged_df = pd.concat([X_enc, array_df], axis=1)

In [10]:
corr_matrix = merged_df.corr('spearman')

In [11]:
import re
overall_useless = []

for i in range(1, 13):
    feature_correlations = corr_matrix[f'price{i}'].drop(f'price{i}')
    sorted_correlations = feature_correlations.abs().sort_values(ascending=False)
    low = sorted_correlations[sorted_correlations<0.2]
    strings = low.index.values
    clean_results = []

    for string in strings:
        clean_string = re.sub(r'_lag\d+', '', string)
        clean_results.append(clean_string)

    unique_results = list(set(clean_results))
    overall_useless.append(unique_results)

In [12]:
flattened_list = [item for sublist in overall_useless for item in sublist]
list(set(flattened_list))

['total_reserve_margin',
 'total_avail',
 'hydro_tng',
 'total_tng',
 'coal_avail_ratio',
 'solar_reserve_margin',
 'coal_reserve_margin',
 'gas_avail_ratio',
 'fossil_fuel_ratio',
 'solar_tng',
 'coal_supply_mix',
 'wind_supply_mix',
 'coal_tng_ratio',
 'hydro_supply_mix',
 'ail',
 'dual_fuel_supply_mix',
 'solar_supply_mix',
 'wind_tng',
 'avail_gen_ratio',
 'hydro_reserve_margin',
 'dual_fuel_tng',
 'demand_supply_ratio',
 'storage_supply_mix',
 'solar_avail',
 'coal_avail',
 'gas_avail',
 'energy_reserve_margin',
 'coal_tng',
 'dual_fuel_avail',
 'storage_tng',
 'wind_reserve_margin',
 'storage_reserve_margin']

In [13]:
# Usefull features
list(set(price_old_df.columns) - set(flattened_list))

['other_tng',
 'gas_tng_ratio',
 'renewable_energy_ratio',
 'other_avail',
 'other_reserve_margin',
 'gas_reserve_margin',
 'storage_avail',
 'gas_tng',
 'hydro_avail',
 'wind_avail',
 'other_supply_mix',
 'renewable_energy_penetration',
 'gas_price',
 'gas_supply_mix',
 'relative_gas_reserve',
 'price',
 'load_on_gas_reserve',
 'gas_cost']

In [19]:
corr_matrix_pearson = merged_df.corr('pearson')

In [20]:
import re
overall_useless = []

for i in range(1, 13):
    feature_correlations = corr_matrix_pearson[f'price{i}'].drop(f'price{i}')
    sorted_correlations = feature_correlations.abs().sort_values(ascending=False)
    low = sorted_correlations[sorted_correlations<0.2]
    strings = low.index.values
    clean_results = []

    for string in strings:
        clean_string = re.sub(r'_lag\d+', '', string)
        clean_results.append(clean_string)

    unique_results = list(set(clean_results))
    overall_useless.append(unique_results)

In [21]:
flattened_list = [item for sublist in overall_useless for item in sublist]
list(set(flattened_list))

['total_reserve_margin',
 'other_avail',
 'total_avail',
 'wind_avail',
 'hydro_tng',
 'total_tng',
 'coal_avail_ratio',
 'solar_reserve_margin',
 'coal_reserve_margin',
 'gas_avail_ratio',
 'solar_tng',
 'other_supply_mix',
 'coal_supply_mix',
 'renewable_energy_penetration',
 'storage_avail',
 'wind_supply_mix',
 'coal_tng_ratio',
 'hydro_supply_mix',
 'ail',
 'dual_fuel_supply_mix',
 'solar_supply_mix',
 'renewable_energy_ratio',
 'other_reserve_margin',
 'wind_tng',
 'avail_gen_ratio',
 'hydro_avail',
 'hydro_reserve_margin',
 'dual_fuel_tng',
 'demand_supply_ratio',
 'solar_avail',
 'storage_supply_mix',
 'coal_avail',
 'gas_avail',
 'energy_reserve_margin',
 'coal_tng',
 'dual_fuel_avail',
 'storage_tng',
 'price',
 'wind_reserve_margin',
 'storage_reserve_margin']

In [22]:
# Usefull features
list(set(price_old_df.columns) - set(flattened_list))

['other_tng',
 'gas_tng_ratio',
 'fossil_fuel_ratio',
 'gas_reserve_margin',
 'gas_tng',
 'relative_gas_reserve',
 'gas_supply_mix',
 'gas_price',
 'load_on_gas_reserve',
 'gas_cost']